<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_1107softmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ソフトマックス関数解題

- data: 2022_1107


In [ ]:
import numpy as np
import sys
import scipy
#from scipy.special import softmax
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

def Softmax(x:np.array,
            beta:float=1.0):
    return np.exp(x/beta)/np.exp(x/beta).sum()

data = np.array(np.arange(5))[::-1]
betas = [0.1, 0.2, 0.4, 0.8, 
         1, 2, 4, 8,
         16, 32, 64, 128]

plt.figure(figsize=(12,5))
for i, beta in enumerate(betas):
    data = np.array(np.arange(5))[::-1]
    output = Softmax(data, float(beta))
    plt.subplot(3,4,(i+1))
    plt.ylim(0,1)
    plt.bar(range(len(output)), output)
    plt.title(f'beta:{beta}')
    #print(output)
plt.show()


In [ ]:
import numpy as np
np.set_printoptions(precision=5)

def kWTA(X:np.ndarray=np.random.randn(5), # 入力層
         k:int=1,                         # 勝者の数
         alpha = 0.,                      # 崩壊率
         _lambda:float=1.0,               # 掃除層からのフィードバックかかるハイパーパラメータ
                                          # lambda は Python の予約語で使えないため _labmda にした
         n2k_limit:float=0.000001,
         n2k_threshold:float=1.0,
         max_t:int=50000,
         verbose=False)->dict:
    """WTA の実装 """
    
    max_u = np.argmax(X) # 初期状態での最大活性値を与えるユニット番号を保持しておく
    if alpha == 0:       # 崩壊率，ユニット数が多いほど，小さい数値にした方が安定するので，こうしてみた。
        alpha = 10 ** -(np.log10(X.shape[0])+5) 

    n = float(len(X))
    n2k = n - (2. * k)

    for t in range(max_t):                # max_t を上限として反復
        Y = np.tanh(X)                    # 出力の計算
        n_winners = ((Y > 0) * 1.).sum()  # 勝者数，出力が正のユニット数を計測
        if ((n_winners == k)):            # 勝者数が条件どおりなら終了
            break

        old_n2k = n2k                     # 内部状態の更新，すなわち，拘束条件の計算
        n2k = np.sum(Y) + (n - (2. * k))
        #if np.abs(n2k - old_n2k) < n2k_limit: break
        
        X += - alpha * Y - _lambda * n2k  # 右辺第 1 項は自己回帰，右辺第 2 項は掃除層からのフィードバック
        
        if verbose:
            print(f't:{t} n_winners:{n_winners}, n2k:{n2k:.3f} old_n2k:{old_n2k:.3f}')
            print(f'X:{X}\n   X:{Y}')
            
    _max_u = np.argmax(Y)                  # 反復計算終了時の最大活性を示すユニット番号
    OK = False if max_u != _max_u or t == max_t else True
    
    return {'Y':Y, 'n_winners':n_winners, 'n2k':n2k, 'time':t, 'OK':OK}

In [ ]:
import numpy as np
import sys
import scipy
from scipy.special import softmax
import matplotlib.pyplot as plt
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

def Softmax(x:np.array,
            beta:float=1.0,
           ):
    d0 = np.exp(x/beta)
    d1 = np.exp(x/beta).sum()
    return d0/d1

data = np.array(np.arange(5))[::-1]
#print(f'data:{data}')
betas = [ 4,  8, 12, 14, 15, 16, 
         20, 24, 25, 26, 27, 28, 
         29, 30, 31, 32, 33, 38, 
         42, 48, 52, 58, 60, 64]

plt.figure(figsize=(20,14))
for i, beta in enumerate(betas):
    data = np.array(np.arange(5))[::-1]
    output = Softmax(data, float(beta))
    plt.subplot(4, 6,(i+1))
    plt.ylim(0,1)
    plt.bar(range(len(output)), output)
    #print(output)
    ret = kWTA(X=output, k=1)
    plt.title(f'beta:{beta}, 反復回数:{ret["time"]}')
plt.show()
